In [1]:
from icevision import models, parsers, show_records, tfms, Dataset, Metric, COCOMetric, COCOMetricType
from pycocotools.coco import COCO
from pycocotools.cocoeval import COCOeval
 
from icevision.imports import *
from icevision.utils import *
from icevision.data import *
from icevision.metrics.metric import *

import numpy as np

import json
import os
from PIL import Image
from pycococreatortools import pycococreatortools

In [2]:
model_type = models.torchvision.mask_rcnn
backbone = model_type.backbones.resnet34_fpn()

In [3]:
mount_path = "/root/data/"

In [5]:
import json

with open(f"{mount_path}/cerulean-v2-no-context-tiled-512-infra-coincident-only/instances_tiled_cerulean_v2_no_context.json", 'r') as JSON:
    json_dict = json.load(JSON)

In [6]:
list(json_dict.keys())

['info', 'licenses', 'images', 'annotations', 'categories']

In [8]:
json_dict['categories'][0]['id'] = 1
json_dict['categories'][1]['id'] = 2
json_dict['categories'][2]['id'] = 3
json_dict['categories'][3]['id'] = 4
json_dict['categories'][4]['id'] = 5
json_dict['categories'][5]['id'] = 6

In [9]:
json_dict['categories']

[{'supercategory': 'slick', 'id': 1, 'name': 'infra_slick'},
 {'supercategory': 'slick', 'id': 2, 'name': 'natural_seep'},
 {'supercategory': 'slick', 'id': 3, 'name': 'coincident_vessel'},
 {'supercategory': 'slick', 'id': 4, 'name': 'recent_vessel'},
 {'supercategory': 'slick', 'id': 5, 'name': 'old_vessel'},
 {'supercategory': 'slick', 'id': 6, 'name': 'ambiguous'}]

In [7]:
ids = []
for i in range(len(json_dict['annotations'])):
    ids.append(json_dict['annotations'][i]['category_id'])
    
print(set(ids))
    

{1, 2, 3, 4, 5, 6}


In [10]:
with open(f"{mount_path}/cerulean-v2-no-context-tiled-512-infra-coincident-only/instances_tiled_cerulean_v2_no_context_updt.json", 'w') as fp:
    json.dump(json_dict, fp)

In [6]:
parser = parsers.COCOMaskParser(annotations_filepath=f"{mount_path}/tile-cerulean-v2-partial-with-context/instances_Tiled Cerulean Dataset V2.json", img_dir=f"{mount_path}/tile-cerulean-v2-partial-with-context/tiled_images")


In [7]:
train_records, valid_records = parser.parse()

  0%|          | 0/3883 [00:00<?, ?it/s]

INFO     - Autofixing records | icevision.parsers.parser:parse:122


  0%|          | 0/3018 [00:00<?, ?it/s]

In [9]:
class_map = {
    "Infrastructure": 1,
    "Natural Seep": 2,
    "Coincident Vessel": 3,
    "Recent Vessel": 4,
    "Old Vessel": 5,
    "Ambiguous": 6,
    "Hard Negatives": 0,
}

class_map

{'Infrastructure': 1,
 'Natural Seep': 2,
 'Coincident Vessel': 3,
 'Recent Vessel': 4,
 'Old Vessel': 5,
 'Ambiguous': 6,
 'Hard Negatives': 0}

In [42]:
x=show_records(train_records[0:15], ncols=3, class_map=class_map)

Normalizing is best practice and necessary for icevision to properly display predicition results

In [10]:
train_tfms = tfms.A.Adapter(
    [
        tfms.A.Normalize(),
    ]
)

In [11]:
valid_tfms = tfms.A.Adapter([*tfms.A.resize_and_pad(size=512), tfms.A.Normalize()])

sourced from: https://airctic.com/0.8.1/getting_started_instance_segmentation/

In [12]:
train_ds.records.autofix??

Object `train_ds.records.autofix` not found.


In [13]:
__all__ = ["IoUMetric", "IoUMetricType"]


class IoUMetricType(Enum):
    """Available options for `COCOMetric`."""

    bbox = "bbox"
    mask = "segm"
    keypoint = "keypoints"


class IoUMetric(Metric):
    """Wrapper around [cocoapi evaluator](https://github.com/cocodataset/cocoapi)
    Calculates average precision.
    # Arguments
        metric_type: Dependent on the task you're solving.
        print_summary: If `True`, prints a table with statistics.
        show_pbar: If `True` shows pbar when preparing the data for evaluation.
    """

    def __init__(
        self,
        metric_type: IoUMetricType = IoUMetricType.mask,
        iou_thresholds: Optional[Sequence[float]] = None,
        print_summary: bool = False, 
        show_pbar: bool = True,
    ):
        self.metric_type = metric_type
        self.iou_thresholds = iou_thresholds
        self.print_summary = print_summary
        self.show_pbar = show_pbar
        self._records, self._preds = [], []

    def _reset(self):
        self._records.clear()
        self._preds.clear()

    def accumulate(self, preds):
        for pred in preds:
            self._records.append(pred.ground_truth)
            self._preds.append(pred.pred)

    def finalize(self) -> Dict[str, float]:
        with CaptureStdout():
            coco_eval = create_coco_eval(
                records=self._records,
                preds=self._preds,
                metric_type=self.metric_type.value,
                iou_thresholds=self.iou_thresholds,
                show_pbar=self.show_pbar,
            )
            coco_eval.evaluate()
            coco_eval.accumulate()

        with CaptureStdout(propagate_stdout=self.print_summary):
            coco_eval.summarize()

        stats = coco_eval.stats
        ious = coco_eval.ious

        ious_l = []
        for iou in ious.values():
            if isinstance(iou, np.ndarray):
                iou = iou.tolist()
            else:
                iou = iou
            ious_l.append(iou)
        
        flat_ious_l = [item for sublist in ious_l for item in sublist]
        if len(flat_ious_l) == 0:
            flat_ious_l.append([0])
        flat_ious_l = [item for items in flat_ious_l for item in items]
        ious_avg = np.array(flat_ious_l).mean()
        ious_min = np.array(flat_ious_l).min()
        ious_max = np.array(flat_ious_l).max()
        
        logs = {
            #"Min IoU area=all": ious_min,
            #"Max IoU area=all": ious_max,
            "Avg. IoU area=all": ious_avg, 
        }
        self._reset()
        
        
        return logs

In [14]:
train_ds = Dataset(train_records, train_tfms)
valid_ds = Dataset(valid_records, valid_tfms)

train_dl = model_type.train_dl(train_ds, batch_size=8, num_workers=6, shuffle=True) # adjust num_workers for your processor count
valid_dl = model_type.valid_dl(valid_ds, batch_size=8, num_workers=6, shuffle=False)

infer_dl = model_type.infer_dl(valid_ds, batch_size=8, shuffle=False)

model = model_type.model(backbone=backbone(pretrained=False), num_classes=len(parser.class_map))

metrics = [IoUMetric(metric_type=IoUMetricType.mask)]

learn = model_type.fastai.learner(dls=[train_dl, valid_dl], model=model, metrics=metrics)


#lr = learn.lr_find()

/root/miniconda3/envs/fastai2/lib/python3.9/site-packages/torch/utils/data/dataloader.py:478: UserWarning: This DataLoader will create 6 worker processes in total. Our suggested max number of worker in current system is 4, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg(


In [15]:
#lr

The suggested learning rate makes getting to higher confidence predictions take too long. We picked the learning rate arbitrarily below to speed up getting to losses closer to .5 instead of greater than 1. 

In [ ]:
#learn.fine_tune(30,2.511886486900039e-03)
learn.fine_tune(30, 3e-3) #, freeze_epochs=2)

epoch,train_loss,valid_loss,IoUMetric,time
0,19.346903,8.839891,0.100018,02:00


/root/miniconda3/envs/fastai2/lib/python3.9/site-packages/torch/functional.py:445: UserWarning: torch.meshgrid: in an upcoming release, it will be required to pass the indexing argument. (Triggered internally at  /opt/conda/conda-bld/pytorch_1640811803361/work/aten/src/ATen/native/TensorShape.cpp:2157.)
  return _VF.meshgrid(tensors, **kwargs)  # type: ignore[attr-defined]


  0%|          | 0/604 [00:00<?, ?it/s]

  0%|          | 0/604 [00:00<?, ?it/s]

epoch,train_loss,valid_loss,IoUMetric,time
0,0.725135,0.750635,0.000000,04:21
1,0.738893,0.728385,0.072215,04:12
2,0.709108,0.756090,0.053233,04:09
3,0.683559,0.726526,0.008214,04:08
4,0.675403,0.708314,0.014987,04:11
5,0.637567,0.671817,0.085851,04:32
6,0.628666,0.639446,0.085657,04:47
7,0.590832,0.623696,0.097203,04:59


...

  0%|          | 0/604 [00:00<?, ?it/s]

  0%|          | 0/604 [00:00<?, ?it/s]

  0%|          | 0/604 [00:00<?, ?it/s]

  0%|          | 0/604 [00:00<?, ?it/s]

  0%|          | 0/604 [00:00<?, ?it/s]

a TODO is to debug the COCOMetric, it should not be -1 given that we are now acheiving detections that intersect with groundtruth.

In [14]:
print(f"approximate time to train 30 epochs in minutes: {25*30/60}")


approximate time to train 30 epochs in minutes: 12.5


The predictions above .7 confidence that roughly line up with groundtruth demonstrates that icevision-trained models can produce predictions that look like they are headed in the correct direction, even for an imperfect training set.

In [15]:
model_type.show_results??

Signature:
model_type.show_results(
    model: torch.nn.modules.module.Module,
    dataset: icevision.data.dataset.Dataset,
    detection_threshold: float = 0.5,
    mask_threshold: float = 0.5,
    num_samples: int = 6,
    ncols: int = 3,
    denormalize_fn: Optional[<built-in function callable>] = <function denormalize_imagenet at 0x7f4f0b195040>,
    show: bool = True,
    device: Optional[torch.device] = None,
) -> None
Docstring: <no docstring>
Source:   
def show_results(
    model: nn.Module,
    dataset: Dataset,
    detection_threshold: float = 0.5,
    mask_threshold: float = 0.5,
    num_samples: int = 6,
    ncols: int = 3,
    denormalize_fn: Optional[callable] = denormalize_imagenet,
    show: bool = True,
    device: Optional[torch.device] = None,
) -> None:
    return base_show_results(
        predict_fn=predict,
        model=model,
        dataset=dataset,
        num_samples=num_samples,
        ncols=ncols,
        denormalize_fn=denormalize_fn,
        show=show,

In [ ]:
x = model_type.show_results(model, valid_ds, detection_threshold=.6) 
plt.savefig("inference_results.png")

In [17]:
show_results??

Object `show_results` not found.
